In [3]:
import configparser
import os.path
import re
from sys import stdout,stderr
import bitarray
import math, pprint, argparse, time
from functools import reduce

import joblib
import scipy
from scipy import ndimage, signal, interpolate
import csv
from pylab import *
import numpy as np

# from petsysbuild.petsys import tofpet2b, tofpet2c, fe_power, daqd, config


class args:
	data_dir = "/mnt/sda2/tofpet_data/test_stand_init/"
	config = f"{data_dir}/config.ini"
	output = f"{data_dir}/disc_staircase_"

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [15]:
def update_asic_coinc(filename_pinout, filename_trig):
    # Pinout colums:
    # 0Layer	1preampchannel	2chipID	3channelID	4chipID_coinc	5channelID_coinc	6SIG	7AbsID	8absID_coinc 9hit_det_id	10hit_pair_id	12assembly_id
    pinout=np.loadtxt(filename_pinout, delimiter=",", skiprows=1)    
    coinc_pairs = [[max(a[:2]),min(a[:2]), a[2]] for a in pinout[:,[7,8,10]]]
    coinc_pairs = np.unique(coinc_pairs, axis=0).astype(int)
    # Save coincidence pairs
    np.savetxt("configurations/coinc_pairs.csv",coinc_pairs, delimiter=",", fmt="%d")

    # Open the ASIC trigger file  
    os.system(f"\cp {filename_trig} {filename_trig}.backup")
    with open(filename_trig, 'w', newline='') as f:
        f.write(f"0\t1\tC\n")
        f.write(f"0\t2\tC\n")
        f.write(f"0\t3\tC\n")
        f.write(f"1\t2\tC\n")
        f.write(f"1\t3\tC\n")
        f.write(f"1\t3\tC\n")
        for pair in coinc_pairs:
            f.write(f"{pair[0]+1000}\t{pair[1]+1000}\tC\n")



def update_asic_map(filename_map):
    # Make the region ID unique for each ASIC channel
    # Open the file
    with open(filename_map, 'r') as f:
        output=[]
        for line in f:
            output.append(line.split())    
    os.system(f"\cp {filename_map} {filename_map}.backup")

    # Update the content
    for i in range(len(output)):
        output[i][4] = f"{int(output[i][2])*64+int(output[i][3])+1000}"
        output[i] = reduce(lambda x, y: str(x) + '\t' + str(y), output[i])+"\n"

    # Write the file
    with open(filename_map, 'w', newline='') as f:
        f.writelines(output)    


filename_pinout ="configurations/MATHUSLA teststand pinout - Preamp to ASIC.csv"
filename_map    = args.data_dir + "/map_channel.tsv"
filename_trig   = args.data_dir + "/map_trigger.tsv"
update_asic_map(filename_map)
update_asic_coinc(filename_pinout, filename_trig)